In [1]:
import sys
sys.path.append("/home/pervinco/DL-workspace/NLP/Seq2Seq_Translation")

In [2]:
import time
import math
import torch
import spacy
import torchtext.data as data
import torchtext.vocab as vocab

from torch import nn
from konlpy.tag import Mecab
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader

from data.utils import get_total_data, split_data
from models.model import Seq2Seq, Encoder, Decoder

2023-11-24 21:08:17.219138: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 21:08:17.239624: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 21:08:17.581185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-24 21:08:17.971746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/home/pervinco/Datasets/KORENG"

In [4]:
max_seq_len = 50

In [5]:
ko_tokenizer = Mecab()
en_tokenizer = spacy.load('en_core_web_sm')

In [6]:
def tokenize_ko(text):
    return [tok for tok in ko_tokenizer.morphs(text)]

def tokenize_en(text):
    return [tok.text for tok in en_tokenizer.tokenizer(text)]

In [7]:
def build_vocab(data_iter, tokenizer):
    vocab = build_vocab_from_iterator(map(tokenizer, data_iter), specials=["<pad>", "<sos>", "<eos>", "<unk>"], min_freq=2)
    vocab.set_default_index(vocab['<unk>'])

    return vocab

In [8]:
dataset = get_total_data(data_dir)

src_sentences = dataset[0]
trg_sentences = dataset[1]
print(len(src_sentences), len(trg_sentences))

total_data.csv exist.
1402407 1402407


In [9]:
src_vocabs = build_vocab(src_sentences, tokenize_ko)
trg_vocabs = build_vocab(trg_sentences, tokenize_en)

print(len(src_vocabs), len(trg_vocabs))

114218 102292


In [10]:
def tokenize_and_convert_to_ids(tokenizer, vocab, sentences):
    tokenized_sentences = [tokenizer(sentence) for sentence in sentences]
    
    return [vocab.lookup_indices(sentence) for sentence in tokenized_sentences]

def pad_sequences(sequences, max_len, pad_token_id=0):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) > max_len:
            padded_sequence = sequence[:max_len]
        else:
            padded_sequence = sequence + [pad_token_id] * (max_len - len(sequence))
        padded_sequences.append(padded_sequence)

    return padded_sequences

In [11]:
src_sentences_ids = tokenize_and_convert_to_ids(tokenize_ko, src_vocabs, src_sentences)
trg_sentences_ids = tokenize_and_convert_to_ids(tokenize_en, trg_vocabs, trg_sentences)

src_sentences_ids_padded = pad_sequences(src_sentences_ids, max_seq_len)
trg_sentences_ids_padded = pad_sequences(trg_sentences_ids, max_seq_len)

In [12]:
train_data, valid_data, test_data = split_data(src_sentences_ids_padded, trg_sentences_ids_padded, train_frac=0.8, valid_frac=0.1)
print(len(train_data[0]), len(train_data[1]))
print(len(valid_data[0]), len(valid_data[1]))
print(len(test_data[0]), len(test_data[1]))

1121925 1121925
140240 140240
140242 140242


In [13]:
class TranslationDataset(Dataset):
    def __init__(self, dataset):
        self.src_indices = dataset[0]
        self.trg_indices = dataset[1]

    def __len__(self):
        return len(self.src_indices)

    def __getitem__(self, idx):
        src = torch.tensor(self.src_indices[idx], dtype=torch.long)
        trg = torch.tensor(self.trg_indices[idx], dtype=torch.long)

        src_batch = src.view(-1, src.shape[0])
        trg_batch = trg.view(-1, trg.shape[0])
        
        return src_batch, trg_batch

In [14]:
batch_size = 128

train_dataset = TranslationDataset(train_data)
valid_dataset = TranslationDataset(valid_data)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

In [15]:
for src, trg in train_loader:
    print(src.shape, trg.shape)

    break

torch.Size([128, 1, 50]) torch.Size([128, 1, 50])


In [16]:
INPUT_DIM = len(src_vocabs)
OUTPUT_DIM = len(trg_vocabs)
EMBEDD_DIM = 1024
HIDDEN_DIM = 2048
NUM_LAYERS = 4
ENCODER_DROPOUT = 0.5
DECODER_DROPOUT = 0.5

encoder = Encoder(input_dim=INPUT_DIM, embedd_dim=EMBEDD_DIM, hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS, dropout=ENCODER_DROPOUT).to(device)
decoder = Decoder(output_dim=OUTPUT_DIM, embedd_dim=EMBEDD_DIM, hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS, dropout=DECODER_DROPOUT).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

In [17]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(114218, 1024)
    (rnn): LSTM(1024, 2048, num_layers=4, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(102292, 1024)
    (rnn): LSTM(1024, 2048, num_layers=4, dropout=0.5)
    (fc_out): Linear(in_features=2048, out_features=102292, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [18]:
optimizer = torch.optim.Adam(model.parameters())

trg_pad_idx = trg_vocabs.lookup_indices(["<pad>"])
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx[0])

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, trg = batch[0].to(device), batch[1].to(device) ## [128, 100]
        
        optimizer.zero_grad()
        print(src.shape, trg.shape)
        output = model(src, trg) ## [max_len, batch_size, src_vocab_size], 100, 128, 102292
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [20]:
def evaluate(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, trg = batch[0].to(device), batch[1].to(device)
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [21]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [24]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

torch.Size([128, 1, 50]) torch.Size([128, 1, 50])


AssertionError: LSTM: Expected input to be 2-D or 3-D but received 4-D tensor